In [1]:
# coding: utf-8
%matplotlib inline
import os
import time
import numpy as np
import pandas as pd
import pyabf
import matplotlib.pyplot as plt
from datetime import datetime as dt
from eventsSegments import segment2
from utils import centering, rolling_window
from sklearn.utils import shuffle
from tqdm import tnrange, tqdm_notebook

from keras.optimizers import Adagrad
from tslearn.preprocessing import TimeSeriesScalerMinMax, TimeSeriesScalerMeanVariance
from tslearn.shapelets import ShapeletModel, grabocka_params_to_shapelet_size_dict

Using TensorFlow backend.


In [2]:

def abfreader(abf_file):
    abf = pyabf.ABF(abf_file)
    abf.setSweep(0)
    signal = abf.sweepY

    return signal


def subtract_baseline(s, baseline, random=True):
    len_signal = s.size
    len_baseline = baseline.size
    baseline_mat = rolling_window(baseline, window=len_signal, asteps=1)
    if random:
        num_windows = baseline_mat.shape[0]
        idx = np.random.randint(num_windows, size=10000 if num_windows > 10000 else 1000)
        baseline_mat = baseline_mat[idx, :]
    similarity = np.sum((s.reshape((1, -1)) - baseline_mat)**2, axis=-1)
    min_idx = np.argmin(similarity)
    return s - baseline_mat[min_idx, :]


def longest_baseline(df, signal):
    starts = df.loc[1:, "start"].reset_index(level=0, drop=True)
    ends = df.loc[0:df.shape[0] - 2, "stop"].reset_index(level=0, drop=True)
    base_dwell = starts - ends
    max_idx = base_dwell.idxmax()
    baseline = signal[ends[max_idx]:starts[max_idx]]
    baseline -= baseline.mean()
    return baseline


In [3]:
DIR = "../data/"
# open_current, threshold, analytes, lower_blockade, upper_blockade
AA3_abfs = {"15o15012.abf": (47, 40, "AA3", 0, 100), 
            "15o15023.abf": (49, 40, "AA3", 0, 100)}
GA3_abfs = {"15o18003.abf": (45, 40, "GA3", 0, 100),
            "15o18017.abf": (45, 40, "GA3", 0, 100)}
all_abfs = {**AA3_abfs, **GA3_abfs}

In [4]:
lower_points = 1100
upper_points = 1300
targetLength = upper_points

dfs = []
for filename, infos in tqdm_notebook(all_abfs.items(), total=len(all_abfs), desc="1st loop"):
    _filename = os.path.join(DIR, filename)
    signal = abfreader(_filename)
    startStop = segment2(signal, infos[0], None, 5, infos[1])
    width = np.diff(startStop, axis=-1)
    df = pd.DataFrame({'start': startStop[:, 0],
                       'stop': startStop[:, 1],
                       'width': (width.squeeze() - 20),
                       'filename': [filename] * width.size,
                       'analytes': [infos[2]] * width.size})
    df = df.loc[(df['width'] >= lower_points)&(df['width'] <= upper_points), :]
    df.reset_index(level=0, drop=True, inplace=True)
    baseline = longest_baseline(df, signal)
    
    level = []
    segments = []
    raws = []
    drops = []
    for idx in tqdm_notebook(df.index, total=df.shape[0], desc='2nd loop'):
        start = df.loc[idx, 'start']
        stop = df.loc[idx, 'stop']
        length = df.loc[idx, 'width']   
        padding_size = targetLength - length
        
        segment = signal[start + 10:stop - 10]
        segment_mean = segment[50:-50].mean()
        segment = signal[start + 10:stop - 10]
        if segment.min() < 0:
            drops.append(idx)
            continue
        if segment_mean > infos[4] or segment_mean < infos[3]:
            drops.append(idx)
            continue
        
        segment -= segment_mean
        segment = subtract_baseline(segment, baseline)
        segment -= segment.mean()
        segment = np.hstack([segment, np.zeros((padding_size, ))])
        segments.append(segment)
        raws.append(signal[start - 20:start + 20])
        level.append(segment_mean)
    
    df.drop(index=drops, inplace=True)
    df.reset_index(drop=True, inplace=True)
    supply = pd.DataFrame({'raw': raws,
                           'segment': segments,
                           'level': level})
    df = pd.concat([df, supply], axis=1)
    dfs.append(df)

In [5]:
datasets = pd.concat(dfs, axis=0)
datasets.to_hdf("data/nosub250kHz.h5", key="abf", mode="w")
datasets.head(5)

/root/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['filename', 'analytes', 'raw', 'segment']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


,start,stop,width,filename,analytes,raw,segment,level
0,49275,50541,1246,15o15012.abf,AA3,"[47.885254, 48.495605, 48.19043, 49.105957, 49...","[3.75836443901062, 1.9273097515106201, 2.23248...",23.104500
1,378486,379789,1283,15o15012.abf,AA3,"[45.443848, 45.443848, 46.0542, 45.13867, 44.5...","[1.1405439376831055, 0.5301923751831055, -0.08...",22.111444
2,1217885,1219160,1255,15o15012.abf,AA3,"[45.13867, 46.359375, 46.359375, 46.0542, 45.7...","[7.2821502685546875, 7.2821502685546875, 4.840...",21.785723
3,1519286,1520474,1168,15o15012.abf,AA3,"[46.0542, 46.359375, 45.749023, 45.443848, 46....","[4.288922309875488, 3.678570508956909, 3.06821...",21.858732
4,2045561,2046829,1248,15o15012.abf,AA3,"[49.105957, 48.80078, 49.71631, 49.105957, 49....","[7.265286445617676, 8.180813789367676, 6.04458...",21.747799
